In [65]:
from sqlalchemy import create_engine
import pandas as pd

from sqlalchemy.engine import URL

connection_string = "DRIVER={SQL Server};SERVER=localhost;DATABASE=brewery;Truster_Connections=yes"
connection_url = URL.create("mssql+pyodbc", query={"odbc_connect": connection_string})
engine = create_engine(connection_url)

# Views:

#### 1) Вывод каталога товаров, его количества на складе, и цены со скидкой (сложный: таблицы products, stock)

```sql
create view ProductsStockView as
	select product_name,
		product_type,
		prime_price,
		retail_price,
		discount_price, 
		amount 
	from products
		join stock on products.product_id=stock.product_id
GO
```

In [66]:
tableResult = pd.read_sql("SELECT * FROM ProductsStockView", engine)
tableResult

,product_name,product_type,upc_code,prime_price,retail_price,discount_price,amount
0,Балтика 7,Пиво,123456789121,67.0,92.46,83.21,1
1,Балтика 8,Пиво,123456789122,68.0,93.84,93.84,2
2,Балтика 9,Пиво,123456789123,69.0,95.22,76.18,3
3,Strongbow яблоко,Сидр,123456789124,57.0,78.66,78.66,4
4,Strongbow груша,Сидр,123456789125,58.0,80.04,56.03,5


#### 2) Вывод всех заказов и их содержимого (сложный: таблицы orders, order_details, customers)

```sql
create view OrdersOrderDetailsCustomersView as
select orders.order_id,
	   customer_name,
	   order_date,
	   orders.order_status,
	   products.product_id,
	   product_name,
	   discount_price,
	   amount,
	   total_cost
		from orders 
			join order_details on orders.order_id=order_details.order_id 
			join products on order_details.product_id=products.product_id
			join customers on orders.customer_id=customers.customer_id
go
```


In [67]:
tableResult = pd.read_sql("SELECT * FROM OrdersOrderDetailsCustomersView", engine)
tableResult

,order_id,customer_name,order_date,order_status,product_id,product_name,amount,price,cost
0,1,Красное&Белое,2022-05-23 14:25:10,pending,1,Балтика 7,1,83.21,83.21
1,1,Красное&Белое,2022-05-23 14:25:10,pending,2,Балтика 8,2,93.84,187.68
2,1,Красное&Белое,2022-05-23 14:25:10,pending,3,Балтика 9,4,76.18,304.72
3,2,Красное&Белое,2004-05-23 14:25:10,processing,2,Балтика 8,2,93.84,187.68
4,2,Красное&Белое,2004-05-23 14:25:10,processing,3,Балтика 9,3,76.18,228.54
5,2,Красное&Белое,2004-05-23 14:25:10,processing,4,Strongbow яблоко,4,78.66,314.64
6,3,РосАл,2004-05-23 14:25:10,pending,1,Балтика 7,5,83.21,416.05
7,3,РосАл,2004-05-23 14:25:10,pending,3,Балтика 9,6,76.18,457.08
8,3,РосАл,2004-05-23 14:25:10,pending,5,Strongbow груша,7,56.03,392.21
9,4,РосАл,2004-05-23 14:25:10,completed,2,Балтика 8,2,93.84,187.68


# Процедуры (все запросы)

## Вывод

---
### 1) Вывод базы товаров для сотрудника завода с количеством товара на складе и себестоимости продукта(сложный)
```sql
create procedure ShowProductsBase 
as
select * from ProductsStockView
```

In [83]:
tableResult = pd.read_sql("exec ShowProductsBase", engine)
tableResult

,product_name,product_type,upc_code,prime_price,retail_price,discount_price,amount
0,Балтика 7,Пиво,123456789121,67.0,92.46,83.21,1
1,Балтика 8,Пиво,123456789122,68.0,93.84,93.84,2
2,Балтика 9,Пиво,123456789123,69.0,95.22,76.18,3
3,Strongbow яблоко,Сидр,123456789124,57.0,78.66,78.66,4
4,Strongbow груша,Сидр,123456789125,58.0,80.04,56.03,5
5,Балтика 0,Пиво,123456789126,60.0,82.80,82.80,0


---
### 2) Вывод каталога товаров для заказчика
```sql
create procedure ShowProductsCatalog 
as
select product_name,
	   product_type,
	   retail_price, 
	   discount_price 
from ProductsStockView
```

In [69]:
tableResult = pd.read_sql("""exec ShowProductsCatalog""", engine)
tableResult

,product_name,product_type,retail_price,discount_price
0,Балтика 7,Пиво,92.46,83.21
1,Балтика 8,Пиво,93.84,93.84
2,Балтика 9,Пиво,95.22,76.18
3,Strongbow яблоко,Сидр,78.66,78.66
4,Strongbow груша,Сидр,80.04,56.03


---
### 3) Получить информацию о недостающих товарах на складе для существующих заказов(сложный)
```sql
create procedure ShowLackProducts
as
select product_name, 
	   abs(amount_on_stock - ordered) as lack_on_stock 
from (select product_name, 
		sum(OrdersOrderDetailsCustomersView.amount) as ordered, 
		stock.amount as amount_on_stock
	from OrdersOrderDetailsCustomersView 
		join stock 
		  on OrdersOrderDetailsCustomersView.product_id=stock.product_id
	  where order_status != 'completed'
	  group by product_name, stock.amount) 
as a
where (amount_on_stock - ordered) < 0
```

In [70]:
tableResult = pd.read_sql("""exec ShowLackProducts""", engine)
tableResult

,product_name,lack_on_stock
0,Балтика 7,11
1,Балтика 8,2
2,Балтика 9,34
3,Strongbow яблоко,10
4,Strongbow груша,2


---
### 4) Вывод информации обо всех заказах(сложный)
```sql
create procedure ShowAllOrders
as
select order_id, 
	   customer_name, 
	   order_date,
	   order_status,
	   total_prime_cost,
	   total_cost 
from Orders 
	join customers 
		on orders.customer_id=customers.customer_id
```

In [71]:
tableResult = pd.read_sql("""exec ShowAllOrders""", engine)
tableResult

,order_id,customer_name,order_date,order_status,total_prime_cost,total_cost
0,1,Красное&Белое,2022-05-23 14:25:10,pending,479.0,575.61
1,2,Красное&Белое,2004-05-23 14:25:10,processing,571.0,730.86
2,3,РосАл,2004-05-23 14:25:10,pending,1155.0,1265.34
3,4,РосАл,2004-05-23 14:25:10,completed,868.0,1121.68
4,5,РосАл,2004-05-23 14:25:10,cancelled,2628.0,3114.18


---
### 5) Вывод информации о своих заказах для заказчика
```sql
create procedure ShowOrdersByCustomer_id
	@customer_id int
as
select order_id,
	order_date,
	order_status, 
	total_cost 
from Orders
where customer_id = @customer_id
```

In [72]:
tableResult = pd.read_sql("""exec ShowOrdersByCustomer_id ?""", engine, params={'2'})
tableResult

,order_id,order_date,order_status,total_cost
0,3,2004-05-23 14:25:10,pending,1265.34
1,4,2004-05-23 14:25:10,completed,1121.68
2,5,2004-05-23 14:25:10,cancelled,3114.18


---
### 6) Вывод информации о своих заказах для заказчика по статусу заказа
```sql
create procedure ShowOrdersByCustomer_idAndOrder_status
	@customer_id int,
	@order_status varchar(30)
as
select order_id,
	order_date,
	order_status, 
	total_cost
from Orders
where customer_id = @customer_id and order_status = @order_status
```

In [73]:
tableResult = pd.read_sql("""exec ShowOrdersByCustomer_idAndOrder_status ?, ?""", engine, params=['2', 'cancelled'])
tableResult

,order_id,order_date,order_status,total_cost
0,5,2004-05-23 14:25:10,cancelled,3114.18


---
### 7) Вывод содержимого заказа
```sql
create procedure ShowOrderDetails
@order_id int
as
select *
	from OrdersOrderDetailsCustomersView
	where order_id = @order_id
```

In [74]:
tableResult = pd.read_sql("""exec ShowOrderDetails ?""", engine, params=['2'])
tableResult

,order_id,customer_name,order_date,order_status,product_id,product_name,amount,price,cost
0,2,Красное&Белое,2004-05-23 14:25:10,processing,2,Балтика 8,2,93.84,187.68
1,2,Красное&Белое,2004-05-23 14:25:10,processing,3,Балтика 9,3,76.18,228.54
2,2,Красное&Белое,2004-05-23 14:25:10,processing,4,Strongbow яблоко,4,78.66,314.64


---
### 8) Вывод информации о заказчиках
```sql
create procedure ShowCustomers
as
select customer_name,
	email,
	phone_number,
	address
from customers
```

In [75]:
tableResult = pd.read_sql("""exec ShowCustomers""", engine)
tableResult

,customer_name,email,phone_number,address
0,Красное&Белое,red_white@mail.ru,88005553535,"ул. Пушкина, д.10"
1,РосАл,rosal@mail.ru,88005553536,"ул. Пушкина, д.11"


---
### 9) Вывод значения прибыли завода за все время(сложный)
```sql
create procedure ShowAllIncome
as
select sum(total_cost - total_prime_cost) as income 
	from Orders
where order_status='completed'
```

In [76]:
tableResult = pd.read_sql("""exec ShowAllIncome""", engine)
tableResult

,income
0,253.68


---
### 10) Вывод значения прибыли завода от конкретного заказчика (сложный)
```sql
create procedure ShowIncomeByCustomer_id
	@customer_id int
as
select sum(total_cost - total_prime_cost) as income 
	from Orders
where order_status='completed' and customer_id=@customer_id
```

In [77]:
tableResult = pd.read_sql("""exec ShowIncomeByCustomer_id ?""", engine, params=['2'])
tableResult

,income
0,253.68


---
### 11) Вывод информации о недостающих товарах на складе для существующих заказов для конкретного заказа (сложный)
```sql
create procedure ShowLackProductsByOrder_id
	@order_id int
as
select product_name, 
	abs(amount_on_stock - ordered) as lack_on_stock 
from
	(select product_name, 
		sum(OrdersOrderDetailsCustomersView.amount) as ordered, 
		stock.amount as amount_on_stock
	from OrdersOrderDetailsCustomersView 
		join stock on OrdersOrderDetailsCustomersView.product_id=stock.product_id
	where order_status != 'completed' and order_id = @order_id
group by product_name, stock.amount) 
as a
where (amount_on_stock - ordered) < 0
```

In [78]:
tableResult = pd.read_sql("""exec ShowLackProductsByOrder_id ?""", engine, params=['5'])
tableResult

,product_name,lack_on_stock
0,Балтика 7,5
1,Балтика 9,21
2,Strongbow яблоко,6


---
### 12) Вывод информации о недостающих товарах на складе для существующих заказов для конкретного заказчика (сложный)
```sql
create procedure ShowLackProductsByCustomer_name
	@customer_name varchar(30)
as
select product_name, 
	abs(amount_on_stock - ordered) as lack_on_stock 
from
	(select product_name, 
		sum(OrdersOrderDetailsCustomersView.amount) as ordered, 
		stock.amount as amount_on_stock
	from OrdersOrderDetailsCustomersView join stock on OrdersOrderDetailsCustomersView.product_id=stock.product_id
	where order_status != 'completed' and customer_name = @customer_name
	group by product_name, stock.amount) 
as a
where (amount_on_stock - ordered) < 0
```

In [79]:
tableResult = pd.read_sql("""exec ShowLackProductsByCustomer_name ?""", engine, params=['РосАл'])
tableResult

,product_name,lack_on_stock
0,Балтика 7,10
1,Балтика 9,27
2,Strongbow яблоко,6
3,Strongbow груша,2


---
### 13) Вывод информации о сотрудниках завода(сложный)
```sql
create procedure ShowWorkers
as
select email, 
	username, 
	first_name, 
	second_name 
from workers 
	join users on workers.user_id=users.user_id
```

In [80]:
tableResult = pd.read_sql("""exec ShowWorkers""", engine)
tableResult

,email,username,first_name,second_name
0,v.pupkin@brewery.com,v.pupkin,Василий,Пупкин
1,n.pechkin@brewery.com,n.peckin,Николай,Печкин


## Вставка

### 1) Вставка товара в базу товара
```sql
create procedure InsertProduct
	@product_name varchar(30),
	@product_type varchar(30),
	@upc_code varchar(12),
	@prime_price decimal(8, 2)
as
INSERT INTO products(product_name, product_type, upc_code, prime_price) 
	VALUES (@product_name, @product_type, @upc_code, @prime_price)
GO
```

In [84]:
with engine.begin() as conn:
	conn.execute("exec InsertProduct ?, ?, ?, ?", 'Балтика 0', 'Пиво', '123456789126', 60)
tableResult = pd.read_sql("exec ShowProductsBase", engine)
tableResult


,product_name,product_type,upc_code,prime_price,retail_price,discount_price,amount
0,Балтика 7,Пиво,123456789121,67.0,92.46,83.21,1
1,Балтика 8,Пиво,123456789122,68.0,93.84,93.84,2
2,Балтика 9,Пиво,123456789123,69.0,95.22,76.18,3
3,Strongbow яблоко,Сидр,123456789124,57.0,78.66,78.66,4
4,Strongbow груша,Сидр,123456789125,58.0,80.04,56.03,5
5,Балтика 0,Пиво,123456789126,60.0,82.80,82.80,0
